# Live Plotting
(back to overview [offline](../Main.ipynb),[online](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/Main.ipynb))


[read on nbviewer](https://nbviewer.jupyter.org/github/QCoDeS/Qcodes/tree/master/docs/examples/plotting/live_plotting.ipynb)

## TL;DR
Enable live plotting by specifying 'subscripition.default_subscriber' in your qcodesrc.json.

## Introduction
Live plotting in qcodes can be achieved by sending the acquired data to other applications. For this an adapter for the specific application is necessary. These adapter register as subscribers to the qcodes dataset and expose a callback that gets called on regular intervals that can be specified.

## Live plotting with plottr as an example
An example of a live plotting application is for example [*plottr*](https://github.com/kouwenhovenlab/plottr). After installing *plottr* you can use it by simply setting `subscription.default_subscribers` to `["Plottr"]` in your `qcodesrc.json` file or alternatively by updating your qcodes config for your current session:

In [1]:
import qcodes
qcodes.config.subscription.default_subscribers = ["Plottr"]

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-halakh\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-halakh\.qcodes\logs\200415-19660-qcodes.log


With this simple change you can run all your notebooks without any changes and live data will be streamed to plottr. For that let us consider the simple example adapted from the dataset docs:

In [2]:
%matplotlib notebook
import numpy as np
from time import sleep
from qcodes import find_or_create_instrument, initialise_database, new_experiment, Measurement
from qcodes.tests.instrument_mocks import DummyInstrument

In [3]:
dac = find_or_create_instrument(DummyInstrument, 'dac', gates=['ch1'], recreate=True)
dmm = find_or_create_instrument(DummyInstrument, 'dmm', gates=['v1'], recreate=True)


In [4]:
# dmm_parameter simulate a physical signal, in this case an exponentially
# decaying signal 

class dmm_parameter(qcodes.Parameter):
    def __init__(self, name):
        super().__init__(name)
        self.ed = self.exponential_decay(5, 0.2)
        next(self.ed)

    def get_raw(self):
        """
        This method is automatically wrapped to
        provide a ``get`` method on the parameter instance.
        """
        val = self.ed.send(dac.ch1())
        next(self.ed)
        return val

    @staticmethod
    def exponential_decay(a: float, b: float):
        """
        Yields a*exp(-b*x) where x is put in
        """
        x = 0
        while True:
            x = yield
            yield a*np.exp(-b*x) + 0.02*a*np.random.randn()
            
    
dmm.v1 = dmm_parameter('dmm_v1')

In [5]:
initialise_database()
new_experiment(name='tutorial_exp', sample_name="no sample")

tutorial_exp#no sample#58@C:\Users\a-halakh\myData1.db
------------------------------------------------------

Running the next cell should render the data in the live plotting front end:

In [6]:
meas = Measurement()
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone
# make live plotting faster:
meas.write_period = 0.1

with meas.run() as datasaver:      
    for set_v in np.linspace(0, 25, 100):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v))
        # we added some extra sleep so that the live plotting can be observed
        sleep(0.1)

Starting experimental run with id: 247. 


AttributeError: module 'plottr.data.qcodes_dataset' has no attribute 'QcodesDatasetSubscriber'